Download the training and testing datasets to our local directory.

In [642]:
import requests
import csv

urls = {
        'X_train' : "https://s3.amazonaws.com/drivendata/data/7/public/4910797b-ee55-40a7-8668-10efd5c1b960.csv",
        'y_train' : "https://s3.amazonaws.com/drivendata/data/7/public/0bf8bc6e-30d0-4c50-956a-603fc693d966.csv",
        'X_test' : "https://s3.amazonaws.com/drivendata/data/7/public/702ddfc5-68cd-4d1d-a0de-f5f566f76d91.csv",
        'y_test' : "https://s3.amazonaws.com/drivendata/data/7/public/SubmissionFormat.csv"
        }

for i in urls:
    r = requests.get(urls[i])

    text = r.iter_lines()

    reader = csv.reader(text, delimiter=',')

    mylist = list(reader)

    with open(str(i)+'.csv', 'wb') as fp:
        a = csv.writer(fp, delimiter=',')
        data = mylist
        a.writerows(data)

Load our train and test datasets into pandas.

In [832]:
import pandas as pd
import numpy as np

In [840]:
X_train = pd.read_csv('X_train.csv')
y_train = pd.read_csv('y_train.csv')
X_test = pd.read_csv('X_test.csv')

Let's inspect our target variable:

In [841]:
del y_train['id']

In [842]:
y_train['status_group'].value_counts(normalize = True)

functional                 0.543081
non functional             0.384242
functional needs repair    0.072677
Name: status_group, dtype: float64

In [831]:
for i in X_train.columns:
    print i,',' ,'{:.1%}'.format(np.mean(X_train[i].isnull())),'nulls',',',type(X_train[i][0]), X_train[i].nunique()

gps_height , 0.0% nulls , <type 'numpy.float64'> 2441
longitude , 0.0% nulls , <type 'numpy.float64'> 57519
latitude , 0.0% nulls , <type 'numpy.float64'> 57520
population , 0.0% nulls , <type 'numpy.float64'> 1059
public_meeting , 0.0% nulls , <type 'numpy.float64'> 2
permit , 0.0% nulls , <type 'numpy.float64'> 2
construction_year , 0.0% nulls , <type 'numpy.int64'> 55
year_recorded , 0.0% nulls , <type 'numpy.int64'> 5
month_recorded , 0.0% nulls , <type 'numpy.int64'> 12
funder__0 , 0.0% nulls , <type 'numpy.float64'> 2
funder__A/co Germany , 0.0% nulls , <type 'numpy.float64'> 2
funder__Aar , 0.0% nulls , <type 'numpy.float64'> 2
funder__Abasia , 0.0% nulls , <type 'numpy.float64'> 2
funder__Abd , 0.0% nulls , <type 'numpy.float64'> 2
funder__Abdul , 0.0% nulls , <type 'numpy.float64'> 2
funder__Aco/germany , 0.0% nulls , <type 'numpy.float64'> 2
funder__Acord , 0.0% nulls , <type 'numpy.float64'> 2
funder__Acord Ngo , 0.0% nulls , <type 'numpy.float64'> 2
funder__Acra , 0.0% null

In [459]:
X_train.describe()

,id,amount_tsh,gps_height,longitude,latitude,num_private,region_code,district_code,population,construction_year
count,59400.000000,59400.000000,59400.000000,59400.000000,5.940000e+04,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000
mean,37115.131768,317.650385,668.297239,34.077427,-5.706033e+00,0.474141,15.297003,5.629747,179.909983,1300.652475
std,21453.128371,2997.574558,693.116350,6.567432,2.946019e+00,12.236230,17.587406,9.633649,471.482176,951.620547
min,0.000000,0.000000,-90.000000,0.000000,-1.164944e+01,0.000000,1.000000,0.000000,0.000000,0.000000
25%,18519.750000,0.000000,0.000000,33.090347,-8.540621e+00,0.000000,5.000000,2.000000,0.000000,0.000000
50%,37061.500000,0.000000,369.000000,34.908743,-5.021597e+00,0.000000,12.000000,3.000000,25.000000,1986.000000
75%,55656.500000,20.000000,1319.250000,37.178387,-3.326156e+00,0.000000,17.000000,5.000000,215.000000,2004.000000
max,74247.000000,350000.000000,2770.000000,40.345193,-2.000000e-08,1776.000000,99.000000,80.000000,30500.000000,2013.000000


In [460]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

## Variable Clean-up

### Variables to delete

In [839]:
def removal():
    # id: we drop the id column because it is not a useful predictor.
    # 'amount_tsh' is mostly blank - delete
    # construction_year: we will NOT YET delete this column since ~35% of the values are zeros.
    # wpt_name: not useful, delete (too many values)
    # subvillage: too many values, delete
    # scheme_name: this is almost 50% nulls, so we will delete this column
    # num_private: we will delete this column because ~99% of the values are zeros.
    # region: drop this b/c is seems very similar to region_code, though not 100% sure about this one!
    z = ['id', 'amount_tsh', 'num_private', 'wpt_name', 
          'subvillage', 'scheme_name', 'region', 'extraction_type_group',
         'payment', 'quality_group', 'quantity_group', 'source_type', 'waterpoint_type_group', 'ward']
    for i in z:
        del X_train[i]
        del X_test[i]
    return

construction year: let's try filling in the nulls with a given value

In [789]:
def construction():
    for i in [X_train, X_test]:
        i['construction_year'].fillna(X_train['construction_year'].mean(), inplace=True)

date_recorded: this might be a useful variable for this analysis, although the year itself would be useless in a practical scenario moving into the future. We will convert this column into a datetime, and we will also create 'year_recorded' and 'month_recorded' columns just in case those levels prove to be useful. A visual inspection of both casts significant doubt on that possibility, but we'll proceed for now. We will delete date_recorded itself, since random forest cannot accept datetime

In [527]:
def dates():
    for i in [X_train, X_test]:
        i['date_recorded'] = pd.to_datetime(i['date_recorded'])
        i['date_recorded'] = pd.to_datetime(i['date_recorded'])
        i['year_recorded'] = i['date_recorded'].apply(lambda x: x.year)
        i['month_recorded'] = i['date_recorded'].apply(lambda x: x.month)
        del i['date_recorded']
    return

`gps_height`, `latitude`, `longitude`

In [603]:
def locs():
    trans = ['longitude', 'latitude', 'gps_height', 'population']
    for i in [X_train, X_test]:
        i.loc[i.longitude == 0, 'latitude'] = 0
        for z in trans:
            i[z].replace(0., np.NaN, inplace = True)
            data = X_train.groupby(['district_code'])[z]
            i[z] = data.transform(lambda x: x.fillna(x.mean()))
        i['gps_height'].fillna(X_train['gps_height'].mean(), inplace = True)
        X_train['population'].fillna(X_train['population'].mean(), inplace = True)
    return

In [538]:
# public_meeting: we will fill the nulls as 'False'
# permit: we will fill the nulls as 'False'

def bools():
    z = ['public_meeting', 'permit']
    for i in z:
        X_train[i].fillna(False, inplace = True)
        X_train[i] = X_train[i].apply(lambda x: float(x))
        X_test[i].fillna(False, inplace = True)
        X_test[i] = X_test[i].apply(lambda x: float(x))
    return

string columns

In [480]:
def codes():
    # convert region_code and district_code to string objects, since they are actually categorical variables
    for i in ['region_code', 'district_code']:
        X_train[i] = X_train[i].apply(lambda x: str(x))
        X_test[i] = X_test[i].apply(lambda x: str(x))
    return

In [813]:
def dummies():
    global X_train, X_test
    columns = [i for i in X_train.columns if type(X_train[i].iloc[1]) == str]
    for column in columns:
        good_cols = []
        X_train[column].fillna('NULL', inplace = True)
        dumms = pd.get_dummies(X_train[column], prefix = column+'_')
        for i in dumms.columns:
        #    if chi2_contingency(pd.crosstab(dumms[i], y_train['status_group']))[1] < .001:
            good_cols.append(i)
        good_cols = [i for i in good_cols if i in pd.get_dummies(X_test[column], prefix = column+'_').columns]
        X_train = pd.concat((X_train, pd.get_dummies(X_train[column], prefix = column+'_')[good_cols]), axis = 1)
        X_test = pd.concat((X_test, pd.get_dummies(X_test[column], prefix = column+'_')[good_cols]), axis = 1)
        del X_train[column]
        del X_test[column]
    return

In [778]:
def dummies2():
    global X_train, X_test
    columns = [i for i in X_train.columns if type(X_train[i].iloc[1]) == str]
    #columns = ['basin']
    status = pd.get_dummies(y_train['status_group'])
    for column in columns:
        print 'starting column', column
        func = []
        non_func = []
        repair = []
        X_train[column].fillna('NULL', inplace = True)
        dumms = pd.get_dummies(X_train[column], prefix = column+'_')
        for i in dumms.columns:
            if status[dumms[i] == 1]['functional'].mean() > (status['functional'].mean() + .2):
                func.append(i)
            elif status[dumms[i] == 1]['non functional'].mean() > (status['non functional'].mean() + .2):
                non_func.append(i)
            elif status[dumms[i] == 1]['functional needs repair'].mean() > (status['functional needs repair'].mean() + .2):
                repair.append(i)
        func = [i for i in func if i in pd.get_dummies(X_test[column], prefix = column+'_').columns]
        non_func = [i for i in non_func if i in pd.get_dummies(X_test[column], prefix = column+'_').columns]
        repair = [i for i in repair if i in pd.get_dummies(X_test[column], prefix = column+'_').columns]
        
        for i in [X_train, X_test]:
            if len(func) > 0:
                i['func_'+column] = pd.get_dummies(i[column], prefix = column+'_')[func].max(axis=1)
            if len(non_func) > 0:    
                i['non_func_'+column] = pd.get_dummies(i[column], prefix = column+'_')[non_func].max(axis=1)
            if len(repair) > 0:
                i['repair_'+column] = pd.get_dummies(i[column], prefix = column+'_')[repair].max(axis=1)
        del X_train[column]
        del X_test[column]
    return

In [843]:
removal()
construction()
locs()
dates()
bools()
codes()
dummies()

In [845]:
print np.shape(X_train)
print np.shape(X_test)

(59400, 1845)
(14850, 1845)


In [846]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(criterion='entropy',
                                n_estimators=500,
                                max_features='auto',
                                oob_score=True,
                                random_state=1,
                                n_jobs=-1)
                            
rf.fit(X_train, y_train.values.ravel())

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=500, n_jobs=-1, oob_score=True, random_state=1,
            verbose=0, warm_start=False)

In [847]:
print "%.4f" % rf.oob_score_ 

0.8100


In [848]:
scores = cross_val_score(rf, X_train, y_train.values.ravel())

In [849]:
scores.mean()

0.80242424242424237

In [672]:
predictions = rf2.predict(X_test)
y_test = pd.read_csv('y_test.csv')
pred = pd.DataFrame(predictions, columns = [y_test.columns[1]])
del y_test['status_group']
y_test = pd.concat((y_test, pred), axis = 1)
y_test.to_csv('y_test.csv', sep=",", index = False)

In [674]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

In [850]:
from sklearn.ensemble import AdaBoostClassifier

tree = DecisionTreeClassifier(criterion='entropy', 
                              max_depth=1,
                              random_state=0)

ada = AdaBoostClassifier(base_estimator=tree,
                         n_estimators=100, 
                         learning_rate=.5,
                         random_state=0)

In [851]:
scores = cross_val_score(ada, X_train, y_train.values.ravel())

In [852]:
scores.mean()

0.73193602693602688

In [786]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

param_range = [0.01, 0.1, 1.0, 10.0, 100.0]

param_grid = [{'C': param_range}]

gs = GridSearchCV(estimator=LogisticRegression(random_state=0),
                  param_grid=param_grid,
                  scoring='accuracy',
                  cv=2,
                  n_jobs=-1)

scores = cross_val_score(gs, X_train, y_train.values.ravel(), scoring='accuracy', cv=5)
print('CV accuracy: %.3f +/- %.3f' % (np.mean(scores), np.std(scores)))

CV accuracy: 0.747 +/- 0.004


In [ ]:
from sklearn.svm import SVC

param_range = [0.01, 0.1, 1.0, 10.0, 100.0]

param_grid = [{'C': param_range, 
               'kernel': ['rbf']}]

gs = GridSearchCV(estimator=SVC(),
                  param_grid=param_grid,
                  scoring='accuracy',
                  cv=2,
                  n_jobs=-1)

scores = cross_val_score(gs, X_train, y_train.values.ravel(), scoring='accuracy', cv=5)
print('CV accuracy: %.3f +/- %.3f' % (np.mean(scores), np.std(scores)))